In [ ]:
import rospy
import pickle
import moveit_commander
from sensor_msgs.msg import JointState


In [4]:
with open("/home/hri25-group3/ros/src/safety_perception/joint_snapshot1.pkl", "rb") as f:
        data = pickle.load(f)

In [14]:
data[0]

{'t': 1761213852.0178509,
 'names': ['arm_shoulder_pan_joint',
  'arm_shoulder_lift_joint',
  'arm_elbow_joint',
  'arm_wrist_1_joint',
  'arm_wrist_2_joint',
  'arm_wrist_3_joint'],
 'q': [-3.1371305624591272,
  -2.5746733150877894,
  -1.473508358001709,
  -0.6813049477389832,
  1.5681782960891724,
  -0.3417566458331507]}

In [12]:
import moveit_msgs.msg
rospy.init_node('notebook')
state_pub = rospy.Publisher("notebook_state", moveit_msgs.msg.DisplayRobotState)

/tmp/ipykernel_193925/2419975706.py:3: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  state_pub = rospy.Publisher("notebook_state", moveit_msgs.msg.DisplayRobotState)


In [62]:
s = data[5]['names']
print(s)

['arm_shoulder_pan_joint', 'arm_shoulder_lift_joint', 'arm_elbow_joint', 'arm_wrist_1_joint', 'arm_wrist_2_joint', 'arm_wrist_3_joint']


In [50]:
ds = moveit_msgs.msg.DisplayRobotState()
ds.state.joint_state.name = s['names']
ds.state.joint_state.position = s['q']
state_pub.publish(ds)

In [ ]:
move = moveit_commander.MoveGroupCommander('arm')
move.set_max_velocity_scaling_factor(0.5)

move.set_named_target('ready')
move.go()


for i in range (len(data)-1):
    #start
    js1= JointState()
    js1.name=data[i]['names']
    js1.position=data[i]['q']
    move.set_joint_value_target(js1)
    move.set_max_velocity_scaling_factor(0.5)
    move.go()
    j=i+1
    for j in range (len(data)-1):
        #end
        js2= JointState()
        js2.name=data[i]['names']
        js2.position=data[i]['q']
        move.set_joint_value_target(js2)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        #go back to start
        move.set_joint_value_target(js1)
        move.set_max_velocity_scaling_factor(0.5)
        move.go()
        
            


range(0, 11)

In [ ]:
import sys, pickle
import rospy, moveit_commander

IN_PKL  = "joint_snapshot1.pkl"
OUT_PKL = "good_pairs.pkl"
GROUP   = "arm"
NAMED   = "ready"    # or None
VEL     = 0.5

def to_js(s):
    js = JointState()
    js.name = s["names"]
    js.position = s["q"] 
    return js

def main():
    moveit_commander.roscpp_initialize(sys.argv)
    rospy.init_node("label_pairs_short", anonymous=True)

    with open(IN_PKL, "rb") as f: data = pickle.load(f)

    move = moveit_commander.MoveGroupCommander(GROUP)
    move.set_max_velocity_scaling_factor(VEL)
    
    #for ready
    if NAMED:
        move.set_named_target(NAMED)
        move.go(wait=True)
        move.stop()

    # append to existing file if present
    try:
        with open(OUT_PKL, "rb") as f: good = pickle.load(f)
        if not isinstance(good, list): good = []
    except Exception:
        good = []

    print("Enter=save, x=ignore, q=quit\n")

    for i in range(len(data)-1):
        for j in range(i+1, len(data)):
            start = data[i]
            end = data[j]
            js1, js2 = to_js(start), to_js(end)

            # --- start -> end ---
            move.set_joint_value_target(js1)
            move.go(wait=True)
            move.stop()

            move.set_joint_value_target(js2)
            ok = move.go(wait=True)
            move.stop()

            if not ok:# planning problem
                print(f"skip {i}->{j} (plan failed)")
                continue
            #waiting for keyboard input
            ans = input(f"[{i}->{j}] Enter=save, x=ignore, q=quit > ").strip().lower()
            
            if ans == "q": 
                raise KeyboardInterrupt
            if ans != "x":
                good.append({"dir":"i_to_j","i":i,"j":j,"start":start,"end":end})
                with open(OUT_PKL,"wb") as f: pickle.dump(good,f,pickle.HIGHEST_PROTOCOL)
                print(f"saved ({i}->{j})")

            # --- end -> start ---
            move.set_joint_value_target(js1)
            ok = move.go(wait=True)
            move.stop()
            if not ok:
                print(f"return {j}->{i} failed")
                continue
            ans = input(f"[{j}->{i}] Enter=save, x=ignore, q=quit > ").strip().lower()
            if ans == "q": raise KeyboardInterrupt
            if ans != "x":
                good.append({"dir":"j_to_i","i":j,"j":i,"start":end,"end":start})
                with open(OUT_PKL,"wb") as f: pickle.dump(good,f,pickle.HIGHEST_PROTOCOL)
                print(f"saved ({j}->{i})")

    print(f"done. total saved: {len(good)}")
    moveit_commander.roscpp_shutdown()

